# Convolutional Neural Network

Importing the libraries

In [120]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [121]:
tf.__version__

'2.17.0'

Data Preprocessing

Preprocessing the Training set

In [124]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   rotation_range = 30,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   brightness_range=[0.8, 1.2],
                                   channel_shift_range=0.2   
                                  )

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


Preprocessing the Test set

In [126]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


Building the CNN

Initialising the CNN

In [129]:
cnn = tf.keras.models.Sequential()

Step 1 - Convolution

In [131]:
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

C:\Users\ADITYA KUMAR\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Step 2 - Pooling

In [133]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Adding a second convolutional layer

In [135]:
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.5))

Step 3 - Flattening

In [137]:
cnn.add(tf.keras.layers.Flatten())

Step 4 - Full Connection

In [139]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))

Step 5 - Output Layer

In [141]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Training the CNN

Compiling the CNN

In [144]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
cnn.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
# Training the CNN on the Training set and evaluating it on the Test set
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=0.00001
)
history = cnn.fit(x = training_set, validation_data = test_set, epochs = 25 , callbacks=[lr_scheduler])

Epoch 1/25


C:\Users\ADITYA KUMAR\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 79s 302ms/step - accuracy: 0.5163 - loss: 0.6960 - val_accuracy: 0.6345 - val_loss: 0.6576 - learning_rate: 1.0000e-04
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 81s 319ms/step - accuracy: 0.6123 - loss: 0.6576 - val_accuracy: 0.6680 - val_loss: 0.6082 - learning_rate: 1.0000e-04
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 75s 296ms/step - accuracy: 0.6505 - loss: 0.6274 - val_accuracy: 0.6840 - val_loss: 0.5900 - learning_rate: 1.0000e-04
Epoch 4/25
202/250 ━━━━━━━━━━━━━━━━━━━━ 14s 293ms/step - accuracy: 0.6762 - loss: 0.6014

In [ ]:
# 5. Evaluating the Model
evaluation = cnn.evaluate(test_set)
print(f"Test Loss: {evaluation[0]}")
print(f"Test Accuracy: {evaluation[1]}")

In [ ]:
# Plotting the training and validation accuracy and loss
import matplotlib.pyplot as plt
history_dict = history.history

print("Available keys in history:", history_dict.keys())

plt.figure(figsize=(12, 4))

# Subplot for accuracy
plt.subplot(1, 2, 1)
plt.plot(history_dict['accuracy'], label='Training Accuracy')
plt.plot(history_dict['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Subplot for loss
plt.subplot(1, 2, 2)
plt.plot(history_dict['loss'], label='Training Loss')
plt.plot(history_dict['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

Making a single prediction

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
# import random
# prediction_dog_cat = f'dataset/single_prediction/dog_or_cat{random.randint(1,3)}.jpg'
# test_image = image.load_img(prediction_dog_cat, target_size = (64, 64))
image_path = 'dataset/single_prediction/dog.jpg'
test_image = image.load_img(image_path, target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'Dog'
else:
  prediction = 'Cat'   
# print(prediction_dog_cat)

In [ ]:
from PIL import Image, ImageEnhance, ImageFilter

# Load the image
test_image_display = Image.open(image_path)

# Resize the image to a larger resolution
test_image_display = test_image_display.resize((256, 256))  # Resizing to a larger resolution

# Enhance the sharpness
enhancer = ImageEnhance.Sharpness(test_image_display)
test_image_display = enhancer.enhance(2.0)  # Increase sharpness by a factor of 2

# Enhance the contrast
enhancer = ImageEnhance.Contrast(test_image_display)
test_image_display = enhancer.enhance(1.5)  # Increase contrast by a factor of 1.5

# Enhance the brightness
enhancer = ImageEnhance.Brightness(test_image_display)
test_image_display = enhancer.enhance(1.2)  # Increase brightness by a factor of 1.2

# Enhance the color
enhancer = ImageEnhance.Color(test_image_display)
test_image_display = enhancer.enhance(1.5)  # Increase color saturation by a factor of 1.5

# Apply a filter to reduce noise
test_image_display = test_image_display.filter(ImageFilter.MedianFilter(size=3))

# Display the enhanced image
plt.imshow(test_image_display)
plt.title(f'Prediction is  "{prediction}"')
plt.show()

In [ ]:
print(prediction)

In [ ]:
# Evaluate on test set
test_loss, test_acc = cnn.evaluate(test_set)
print(f"Test accuracy: {test_acc:.4f}")

In [ ]:
# 9. Confusion Matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
Y_pred = cnn.predict(test_set)
y_pred = (Y_pred > 0.5).astype(int).ravel()

cm = confusion_matrix(test_set.classes, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(training_set.class_indices.keys()))
disp.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
# Extract TP, TN, FP, FN from confusion matrix
TN, FP, FN, TP = cm.ravel()

# Calculate accuracy
accuracy = accuracy_score(test_set.classes, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Calculate precision
precision = precision_score(test_set.classes, y_pred)
print(f"Precision: {precision:.4f}")

# Calculate recall
recall = recall_score(test_set.classes, y_pred)
print(f"Recall: {recall:.4f}")

# Calculate F1 score
f1 = f1_score(test_set.classes, y_pred)
print(f"F1 Score: {f1:.4f}")

# Print confusion matrix
print("Confusion Matrix:")
print(cm)